# پیش بینی امتیاز داده شده به یک محصول براساس کامنت های داده شده

In [28]:
from sklearn.preprocessing import normalize, LabelEncoder
from sklearn.cluster import KMeans
from numpy import unique
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as shc
from numpy import where
from sklearn.neighbors import KernelDensity
from scipy.signal import argrelextrema
from scipy.misc import electrocardiogram
from scipy.signal import find_peaks
import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from google_trans_new import google_translator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [29]:
keifiat = pd.read_excel('E:/university/data mining/dataset_2/keifiat.xlsx')
keifiat

,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages
0,3692,ماوس بی‌سیم لاجیتک مدل M325,IT,989472,0,0,verified,\N,NaN,واقعا عالیه. من که ازش خیلی راضیم,NaN,NaN
1,90213,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,3862150,4,1,verified,recommended,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]"
2,59473,یدک پولیشر میکروفایبر مهسان مدل 20119,HW,626843,1,0,verified,not_recommended,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,NaN,NaN
3,120499,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,786887,6,11,verified,no_idea,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,"[""صفحه نمایش پرنور و با کیفیت\r"",""کیفیت بالای ...","[""کیفیت پایین اسپیکر\r"",""حاشیه خالی زیر صفحه ن..."
4,67200,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,854531,19,4,verified,no_idea,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...,"[""ظریف و زیبا و باریک\r"",""بدنه محکم و با دوام""]","[""ظرفیت متوسط""]"
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,549845,شید آباژور آرام مدل SH45/01,TC,4979401,0,0,verified,recommended,بسیار تمیز و زیبا,بسیار تمیز و زیبا...من که خیلی راضیم,"[""کاملا مطابق با عکس\r"",""ساخت بسیار تمیز""]","[""ندارد""]"
99996,784160,کفش مخصوص دویدن مردانه و زنانه آلبرتینی مدل دا...,FA,1066034,7,32,verified,\N,توصیه به خرید میکنم,من این کفش را برای پیاده روی اربعین خریدم از ی...,"[""سبک زیبا ارزان ""]","[""محل عبور بند کفش ها ایراد دارد""]"
99997,820985,واکس کفش ساتل مدل 20 حجم 60 میلی لیتر,FA,568208,0,0,verified,recommended,خوبه و غلیظه,واکس خوبیه..,"[""روغنیه""]","[""ندیدم""]"
99998,678073,کش رنگی مدادچوبی مدل 010 بسته 100,TC,981594,2,1,verified,no_idea,سلام,هیچ کدوم از این رنگهای که توی عکسه داخل بسته ن...,NaN,NaN


In [30]:
keifiat=keifiat.filter(['product_id','comment'])

In [31]:
keifiat

,product_id,comment
0,3692,واقعا عالیه. من که ازش خیلی راضیم
1,90213,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...
2,59473,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...
3,120499,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...
4,67200,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...
...,...,...
99995,549845,بسیار تمیز و زیبا...من که خیلی راضیم
99996,784160,من این کفش را برای پیاده روی اربعین خریدم از ی...
99997,820985,واکس خوبیه..
99998,678073,هیچ کدوم از این رنگهای که توی عکسه داخل بسته ن...


# my laptop isnt very good and couldnt translate and process huge numbers of comments so i had to reduce them

In [35]:
index_names = keifiat[keifiat['product_id'] <890000].index 
# drop these given row 
# indexes from dataFrame 
keifiat.drop(index_names,inplace = True) 

In [36]:
keifiat

,product_id,comment
89445,891045,سلام دوستان این گوشی فوق العاده است ازنظر سخت ...
89485,891034,تحویل بموقع و در دسترس بودن خدمات
89753,891045,گرونه وبه بازار ایران دیر میرسه
89969,891312,این کاپشن از جنس بیرونی فوق العاده مناسب در بر...
90202,890725,عالیه واقعا یکی از بهترین صندلیا میتونم بگم هست
...,...,...
99934,907654,کیفیتش خوبه قیمتش هم مناسبه
99935,907647,کیفیتش خوبه قیمتش هم مناسبه
99936,907634,کیفیتش خوبه قیمتش هم مناسبه
99952,908760,من فیکورس مدلهای اونتوس و وود رو دارم، انصافا ...


# first we translate and then do NLP with python-Text Clustering

In [37]:
trans = google_translator()  

In [38]:
translate_text=[]
for comment in keifiat['comment']:
    txt=trans.translate(comment, lang_src='fa', lang_tgt='en')
    print(txt)
    translate_text.append(txt)  

Hello friends, this phone is great in terms of hardware, very strong, especially the 4200 amp battery, it is also very handy and beautiful, it has a very good camera, it is also very economical. 
Timely delivery and availability of services 
Expensive and late to the Iranian market 
This jacket is made of outer material that is extremely suitable for rain and this quality seems very good to me at this price. 
Alia is really one of the best chairs I can say 
I received it yesterday wonderful 
The best phone in the world right now to play 
Hello friends, the new flagship of Huawei is a high-powered phone with high performance that few people can complain about. There is only one problem that with this currency situation in Iran, it is not possible for most of us to buy it. I suggest those who can afford it to buy it. .  Of course, I hope that the currency problem will be solved by the time this product is released 
I bought this brand twice. I got from customs.  I really say just playing

In [39]:
translate_text

['Hello friends, this phone is great in terms of hardware, very strong, especially the 4200 amp battery, it is also very handy and beautiful, it has a very good camera, it is also very economical. ',
 'Timely delivery and availability of services ',
 'Expensive and late to the Iranian market ',
 'This jacket is made of outer material that is extremely suitable for rain and this quality seems very good to me at this price. ',
 'Alia is really one of the best chairs I can say ',
 'I received it yesterday wonderful ',
 'The best phone in the world right now to play ',
 'Hello friends, the new flagship of Huawei is a high-powered phone with high performance that few people can complain about. There is only one problem that with this currency situation in Iran, it is not possible for most of us to buy it. I suggest those who can afford it to buy it. .  Of course, I hope that the currency problem will be solved by the time this product is released ',
 'I bought this brand twice. I got from c

In [40]:
keifiat['commenten']=translate_text

In [41]:
keifiat['commenten']

89445    Hello friends, this phone is great in terms of...
89485        Timely delivery and availability of services 
89753            Expensive and late to the Iranian market 
89969    This jacket is made of outer material that is ...
90202     Alia is really one of the best chairs I can say 
                               ...                        
99934     Its quality is good and its price is reasonable 
99935     Its quality is good and its price is reasonable 
99936     Its quality is good and its price is reasonable 
99952    I have the Ficors of Ontus and Wood models, in...
99992    We used it, it was good, I was looking for it ...
Name: commenten, Length: 121, dtype: object

In [42]:
scaled_features=keifiat['commenten'].to_numpy()
scaled_features

array(['Hello friends, this phone is great in terms of hardware, very strong, especially the 4200 amp battery, it is also very handy and beautiful, it has a very good camera, it is also very economical. ',
       'Timely delivery and availability of services ',
       'Expensive and late to the Iranian market ',
       'This jacket is made of outer material that is extremely suitable for rain and this quality seems very good to me at this price. ',
       'Alia is really one of the best chairs I can say ',
       'I received it yesterday wonderful ',
       'The best phone in the world right now to play ',
       'Hello friends, the new flagship of Huawei is a high-powered phone with high performance that few people can complain about. There is only one problem that with this currency situation in Iran, it is not possible for most of us to buy it. I suggest those who can afford it to buy it. .  Of course, I hope that the currency problem will be solved by the time this product is relea

# i tried some values for numbers of clusters and realized 10 was okay

In [88]:
vectorizer = TfidfVectorizer(stop_words='english',ngram_range=(1,3))
vectorizer.fit(keifiat['commenten'].values)
feautures=vectorizer.transform(keifiat['commenten'].values)
clust = KMeans(max_iter=100,n_clusters=10,n_init=10,init='k-means++')
clust.fit(feautures)

KMeans(max_iter=100, n_clusters=10)

In [89]:
yhat=clust.predict(feautures)
keifiat['cluster labels']=clust.labels_
keifiat=keifiat.filter(['product_id','commenten','cluster labels'])
keifiat

,product_id,commenten,cluster labels
89445,891045,"Hello friends, this phone is great in terms of...",5
89485,891034,Timely delivery and availability of services,1
89753,891045,Expensive and late to the Iranian market,6
89969,891312,This jacket is made of outer material that is ...,3
90202,890725,Alia is really one of the best chairs I can say,2
...,...,...,...
99934,907654,Its quality is good and its price is reasonable,3
99935,907647,Its quality is good and its price is reasonable,3
99936,907634,Its quality is good and its price is reasonable,3
99952,908760,"I have the Ficors of Ontus and Wood models, in...",0


In [90]:
dat=keifiat[(keifiat['cluster labels']==0)]
dat

,product_id,commenten,cluster labels
92055,895299,I bought this brand twice. I got from customs....,0
92521,890864,I like Somayeh's products,0
94922,900065,"Hi friends, I made this car wash. It had bette...",0
94946,900970,I bought a lot of rhinestones and use them alm...,0
96078,890577,"My size is large, but I bought X large, but it...",0
97299,903515,This model is not very tall and it shows the b...,0
97782,892480,Don't really use it ...,0
98730,890656,This game is the most unique series of black o...,0
99858,908646,Due to the fact that the height of the floors ...,0
99952,908760,"I have the Ficors of Ontus and Wood models, in...",0


In [91]:
dat=keifiat[(keifiat['cluster labels']==1)]
dat

,product_id,commenten,cluster labels
89485,891034,Timely delivery and availability of services,1
92062,894350,"It has a very good appearance, which is great ...",1
92873,895394,Useful for those who are interested in this fi...,1
93268,890656,"God, it's very cool, I think it's very cool, b...",1
93511,890656,Excellent and unique,1
94433,893884,"Hello friends, this phone is great and some pe...",1
94929,900065,I am satisfied with my purchase It has good p...,1
95101,896651,The dough is good and suitable. If you are not...,1
96015,892617,It's very good to say that this is great,1
96092,900851,Very good,1


In [92]:
dat=keifiat[(keifiat['cluster labels']==2)]
dat

,product_id,commenten,cluster labels
90202,890725,Alia is really one of the best chairs I can say,2
91358,894437,The best phone in the world right now to play,2
94159,890656,Hello to those who are in game pro and want to...,2
96241,902418,Hi CC shoes are a 70-year-old brand known in ...,2
97452,903832,Seiko Japan watch should be bought with comple...,2
98800,900428,"Once the phone is dropped, the frame crashes",2


In [93]:
dat=keifiat[(keifiat['cluster labels']==3)]
dat

,product_id,commenten,cluster labels
89969,891312,This jacket is made of outer material that is ...,3
94918,900065,"I bought it from the market, unfortunately, th...",3
94958,900873,"It looks great on clothes in purple, yellow, g...",3
95877,901032,"Despite the beauty and new design, it has a re...",3
95949,892013,"On my eyebrows, which had no effect. Even now,...",3
97276,905661,"For a very good price, I have this Gillette pr...",3
97496,896697,"Exactly what you see in the picture, both the ...",3
99888,897332,"It is plastic, but considering its price, it m...",3
99890,897216,"It is plastic, but considering its price, it m...",3
99931,907669,Its quality is good and its price is reasonable,3


In [94]:
dat=keifiat[(keifiat['cluster labels']==4)]
dat

,product_id,commenten,cluster labels
91292,894711,I received it yesterday wonderful,4
93064,894556,It is an excellent brand. I have several model...,4
93807,890656,The latest COD so far,4
94160,890656,All three parts of the amazing game are zombie...,4
94456,892086,This guard is excellent and free ... In this m...,4
94606,891635,I bought this sewing machine for bloody consum...,4
95882,900305,I got it today. It is made of bronze metal and...,4
96055,903442,I am satisfied with having this power bank and...,4
96112,899851,"It's great, but it's cheaper outside",4
96770,898260,Very stylish and. Very valuable class,4


In [95]:
dat=keifiat[(keifiat['cluster labels']==5)]
dat

,product_id,commenten,cluster labels
89445,891045,"Hello friends, this phone is great in terms of...",5
92198,892534,I bought this refrigerator from the company .....,5
94991,900065,It has a telescopic handle for easy transport....,5
95914,900929,"Although it is made of steel, it is very delic...",5
96233,903089,A combination of modern design on Iranian hand...,5
96773,899888,"Very beautiful and stylish, very special",5
97306,903494,It is very beautiful and has a very good design,5
97404,901162,This power bank is really great in terms of be...,5
98626,890738,It has a really beautiful design. I fell in lo...,5
99733,891504,"Very beautiful and elegant, I just wish he had...",5


In [96]:
dat=keifiat[(keifiat['cluster labels']==6)]
dat

,product_id,commenten,cluster labels
89753,891045,Expensive and late to the Iranian market,6
95954,900652,I bought this product when there was nothing a...,6
97585,906128,I just saw this product here As someone who h...,6
98960,899757,"It is easy to clean, it removes stains and dir...",6


In [97]:
dat=keifiat[(keifiat['cluster labels']==7)]
dat

,product_id,commenten,cluster labels
94696,896621,"I have the s970 model now, it is excellent and...",7
97294,903513,It is very comfortable and has a very good mat...,7
97311,903485,The boot is very high quality It also has ver...,7
97322,903513,It is a very good boot both in terms of design...,7
97347,903494,"In terms of build quality, it is both very goo...",7
98163,897525,"It has a very good material, it fits you very ...",7
99014,903490,I had previously made the same half-boot model...,7
99600,891432,"I ordered two, but when I delivered one, it wa...",7


In [98]:
dat=keifiat[(keifiat['cluster labels']==8)]
dat

,product_id,commenten,cluster labels
96238,903151,Stunning dot with great color,8
97282,903482,The ankle boot is very comfortable and very be...,8
97308,903513,"In a word, it is a very beautiful boot and it ...",8
97314,903495,It has both very good quality and very beautif...,8
97319,903494,It is really strong boot and has a very good q...,8


In [99]:
dat=keifiat[(keifiat['cluster labels']==9)]
dat

,product_id,commenten,cluster labels
91996,893884,"Hello friends, the new flagship of Huawei is a...",9
92260,896520,This book is not suitable for people who do no...,9
93348,893884,If anyone can afford to buy this phone,9
94608,891635,The free arm part of this wheel is very comfor...,9
97514,901875,I hope it will be everyone's part,9
97725,897467,"Before buying, I suggest you compare the logo ...",9
98106,893884,"There is no money Whoever has the money, buy it",9
99242,892894,I have been using this rope for ten months. I ...,9


# here is what each cluster means:
# 4satisfactory
# 1good
# 2good
# 6good
# 7good
# 9good
# 5Very good
# 0excellent
# 3excellent
# 8excellent

# and here are their meanings:
# cluster4->rank:1
# cluster1,2,6,7,9->rank:2
# cluster5->rank:3
# cluster0,3,8->rank:4

In [100]:
cd=[]
for label in keifiat['cluster labels']:
    if(label==8): ll=4
    if(label==0): ll=4
    if(label==3): ll=4
    if(label==5): ll=3
    if(label==1): ll=2
    if(label==2): ll=2
    if(label==6): ll=2
    if(label==7): ll=2
    if(label==9): ll=2
    if(label==4): ll=1
    cd.append(ll)

In [101]:
keifiat['rank']=cd
keifiat

,product_id,commenten,cluster labels,rank
89445,891045,"Hello friends, this phone is great in terms of...",5,3
89485,891034,Timely delivery and availability of services,1,2
89753,891045,Expensive and late to the Iranian market,6,2
89969,891312,This jacket is made of outer material that is ...,3,4
90202,890725,Alia is really one of the best chairs I can say,2,2
...,...,...,...,...
99934,907654,Its quality is good and its price is reasonable,3,4
99935,907647,Its quality is good and its price is reasonable,3,4
99936,907634,Its quality is good and its price is reasonable,3,4
99952,908760,"I have the Ficors of Ontus and Wood models, in...",0,4


In [103]:
keifiat.groupby(by=['rank']).first()

,product_id,commenten,cluster labels
rank,,,
1,894711,I received it yesterday wonderful,4
2,891034,Timely delivery and availability of services,1
3,891045,"Hello friends, this phone is great in terms of...",5
4,891312,This jacket is made of outer material that is ...,3


# done.